# <center> Basics of Mobile Robotics Project -  Report Notebook <center>
### Catarina Pires 346796
    
MA3 student in Mechanical Engineering, specialising in automation & control systems
    
### Dimitri Hollosi 247227 
    
MA3 student in Mechanical Engineering, specialising in automation & control systems
    
### Gonçalo Gomes 
    

    
### Richard Gao 342177

MA1 student in Robotics, specialising in mobile robotics and data science


## General Considerations and  Introduction to the Environment


Here the main code structure and functionalities aim to be detailed so as to gain an understanding in the underlying structure behind the Thymio implementations. Generally speaking, the code can be split into 3 main parts, namely Camera Vision, Global Navigation and Local Navigation. Camera Vision is used to generate the shortest path the thymio should take to go from its current position to a defined goal, taking into consideration existing obstacles, and to localize the thymio on the during run time. Glabal Navigation includes the functions that find the thymio and generate the shortest path from computer vision, as well as the implementation of a PID controller to make the thymio follow the desired path, and the implementation of a Kalman Filter to estimate the Thymio's states from the sensor measurements. The latter includes all aspects related to obstacle avoidance, which was iteratively tuned to obtain desired performance. One important element to consider prior to delving into the implemented functionalities within the code would be the chosen environment setup for the final project implementation. Indeed, it was decided to use one of the team member's lower wardrobe portion so as to benefit from a relatively "stable" environment, which could further simplifiy the calibration, lighting and initial setup process. This environment does however come with its limitations, as this implies that the Thymio must navigate in a restricted and confined space, namely implying that the local navigation must take that into account (i.e walls and corners must be accounted for) and yield a relatively high responsiveness.

### Code Structure

Wait for final main to write about this

# Local Navigation & Obstacle Avoidance

The Local Navigation for this project was implemented as a class in order to be able to benefit from conciceness and structure. Local_Nav takes as an input the asynchronous client, the awaited node and finally the motor speeds. The constructor then defines the "self" variables which are to be used throughout the different methods, which proves to be particularly useful as it is readily able to update the values which are to be sent to the thymio (through the node). 

The main class method is obstacle_avoidance and is structured as follows:

The get_sensor_data(self) method gathers horizontal proximity sensor readings at each timestep, which is "fed" into the analyse_data(self) method. The latter returns a flag = 1 if any of the read values were higher than the lower threshold (iteratively deduced to be worth 2800), at which point the obstacle avoidance manoeuvres are returned. From there, depending on which sensors were triggered, the resulting motor speeds are adjusted by means of a weighted gain and directly updated through the ''' self.node.send_set_variables(adjust_speed) ''' command. It ought to be mentioned that three gains (ranging from low to high) were implemented depending on the intensity of the sensed values. Furthermore, the gains are obtained through and average of the extremity sensors and the "mid-extremity" sensors (the ones in between the front middle and each extremity). This allows to single handedly capture most of the real-case scenarios, as testing showed that the Thymio could be as likely to encounter any of the three following scenarios : 

- Only the extremity sensors being triggered
- Only the mid-extremity sensors being triggered
- A combination of both

Therefore, this implementation of the gains allows some flexbility in the manoeuvering should an obstacle be encountered. Regarding the lower threshold of 2800, this was initially set to be of lower value (i.e 1500) but unfortunately proved to be too  sensitive to the walls in the confined environment that the Thymio operated in. To amend to this, the walls were covered in relatively dark material so as to decrease the sensitivity of the sensors.

Finally, the local navigation is easily implemented in the "main" module as : 

local_nav = LN(client, node, cruising, cruising)

flag_obs = local_nav.analyse_data()

    if flag_obs == 1:
        task = local_nav.obstacle_avoidance2() #trigger obstacle avoidance task depending on if flag detected 
    else:
    ...

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

class Local_Nav:

    def __init__(self,client,node, motor_speed_left,motor_speed_right):
        self.motor_speed_left = motor_speed_left
        self.motor_speed_right = motor_speed_right
        self.client = client
        self.node = node 
        self.sat_speed = 500
        self.lower_threshold = 2800 #lower threshold for proximity sensor values
        self.middle_threshold = 3500
        self.upper_threshold = 4000
        
        #The measures were obtained empirically...
        self.sensor_distances = np.array([15, 9.6, 9, 8.5, 7.5, 7, 6, 5, 4, 3, 2.5, 2, 1.5, 0.5, 0.2, 0])
        self.sensor_measurements = np.array(
            [0, 1186, 1420, 1510, 1715, 1830, 2069, 2250, 2468, 2850, 3155, 3468, 3940, 4355, 4432, 4500])
        self.center_offset = np.array([5.5, 5.5])
        self.sensor_pos_from_center = np.array(
            [[0.9, 9.4], [3.1, 10.5], [5.5, 11.0], [8.0, 10.4], [10.2, 9.3], [2.5, 0], [8.5, 0]]) - self.center_offset
        self.sensor_angles = np.array([120, 105, 90, 75, 60, -90, -90]) * math.pi / 180
        self.thymio_coords = np.array([[0, 0], [11, 0], [11, 8.5], [10.2, 9.3],
                                  [8, 10.4], [5.5, 11], [3.1, 10.5],
                                  [0.9, 9.4], [0, 8.5], [0, 0]]) - self.center_offset

#######################################################################################################################
########################################   Local Navigation Section  ##################################################
#######################################################################################################################

    def get_sensor_data(self):
        self.client.aw(self.node.wait_for_variables({"prox.horizontal"}))
        prox_sens = self.node.v.prox.horizontal
        return  list(prox_sens)

    def analyse_data(self):
        prox_sens_data = self.get_sensor_data()
        check = [i for i in prox_sens_data if i >= self.lower_threshold]
        if any(check):
            flag = 1
            return flag
        else :
            flag = 0
            return flag

    def obstacle_avoidance(self):
        ''' If a flag is detected, then obstacle avoidance will be activated. The flag returns 1 if any of the sensors 
        measured something higher that the self.lower_threshold value. The latter was iteratively tuned to adjust to the
        constraining environment conditions, and could be made smaller if not in a confined space (with white walls...)
        for better results.'''
        
        flag = self.analyse_data()
        obstSpeedGain = [6, 4, 2]
        prox_sens_data = self.get_sensor_data()
        extermity_sens = [prox_sens_data[0], prox_sens_data[4]]
        mid_extermity_sens = [prox_sens_data[1], prox_sens_data[3]]
        middle_sensor = prox_sens_data[2]
        gain_low = [(mid_extermity_sens[0] + extermity_sens[0]) * obstSpeedGain[2] // 200,
                    (mid_extermity_sens[1] + extermity_sens[1]) * obstSpeedGain[2] // 200]
        gain_mid = [(mid_extermity_sens[0] + extermity_sens[0]) * obstSpeedGain[1] // 200,
                (mid_extermity_sens[1] + extermity_sens[1]) * obstSpeedGain[1] // 200]
        gain_high = [(mid_extermity_sens[0] + extermity_sens[0]) * obstSpeedGain[0] // 200,
                (mid_extermity_sens[1] + extermity_sens[1]) * obstSpeedGain[0] // 200]
        if flag == 1:
            if extermity_sens[0] or mid_extermity_sens[0] >= self.lower_threshold:
                adjust_speed = self.motors(self.motor_speed_left + gain_low[0],
                                           -self.motor_speed_right - gain_low[1])
                self.node.send_set_variables(adjust_speed)

            elif extermity_sens[1] or mid_extermity_sens[1] >= self.lower_threshold:
                adjust_speed = self.motors(-self.motor_speed_left - gain_low[0],
                                           +self.motor_speed_right + gain_low[1])
                self.node.send_set_variables(adjust_speed)

            elif self.lower_threshold <= middle_sensor <= self.middle_threshold:
                if mid_extermity_sens[0] <= mid_extermity_sens[1]:
                    adjust_speed = self.motors(-self.motor_speed_left - gain_mid[0],
                                               self.motor_speed_right + gain_mid[1])
                else:
                    adjust_speed = self.motors(self.motor_speed_left + gain_mid[0],
                                                -self.motor_speed_right - gain_mid[1])
                self.node.send_set_variables(adjust_speed)

            elif middle_sensor >= self.lower_threshold and mid_extermity_sens[0] >= self.lower_threshold:
                    adjust_speed = self.motors(-self.motor_speed_left - gain_mid[0],
                                               -self.motor_speed_right - gain_mid[1])
                    if mid_extermity_sens[0] >= self.upper_threshold :
                        adjust_speed = self.motors(-self.motor_speed_left - gain_high[0],
                                                   -self.motor_speed_right - gain_high[1])
                    self.node.send_set_variables(adjust_speed)

            elif middle_sensor >= self.lower_threshold and mid_extermity_sens[1] >= self.lower_threshold:
                    adjust_speed = self.motors(-self.motor_speed_left - gain_mid[0],
                                               -self.motor_speed_right - gain_mid[1])
                    if mid_extermity_sens[0] >= self.upper_threshold :
                        adjust_speed = self.motors(-self.motor_speed_left - gain_high[0],
                                                   - self.motor_speed_right -  gain_high[1])
                    self.node.send_set_variables(adjust_speed)

    def motors(self, motor_speed_left, motor_speed_right):
        if motor_speed_left >= 500:
            return {
                "motor.left.target": [self.sat_speed],
                "motor.right.target": [motor_speed_right],
            }
        elif motor_speed_right >= 500:
            return {
                "motor.left.target": [motor_speed_left],
                "motor.right.target": [self.sat_speed],
            }
        elif motor_speed_left <= -500:
            return {
                "motor.left.target": [-self.sat_speed],
                "motor.right.target": [motor_speed_right],
            }
        elif motor_speed_right <= -500:
            return {
                "motor.left.target": [motor_speed_left],
                "motor.right.target": [-self.sat_speed],
            }
        else:
            return {
                "motor.left.target": [motor_speed_left],
                "motor.right.target": [motor_speed_right],
            }

#######################################################################################################################
####################################   Local Occupancy Graph Generation ###############################################
#######################################################################################################################

    def sensor_val_to_cm_dist(self, sens_val):
        """
        Returns the distance corresponding to the sensor value based
        on the sensor characteristics
        :param val: the sensor value that you want to convert to a distance
        :return: corresponding distance in cm
        """
        if sens_val == 0:
            return np.inf

        f = interp1d(self.sensor_measurements, self.sensor_distances)
        return f(sens_val).item()

    def obstacles_pos_from_sensor_vals(self, sensor_vals):
        """
        Returns a list containing the position of the obstacles
        w.r.t the center of the Thymio robot.
        :param sensor_vals: sensor values provided clockwise starting from the top left sensor.
        :return: numpy.array() that contains the position of the different obstacles
        """
        print(sensor_vals)
        dist_to_sensor = [self.sensor_val_to_cm_dist(x) for x in sensor_vals]
        dx_from_sensor = [d * math.cos(alpha) for (d, alpha) in zip(dist_to_sensor, self.sensor_angles)]
        dy_from_sensor = [d * math.sin(alpha) for (d, alpha) in zip(dist_to_sensor, self.sensor_angles)]
        obstacles_pos = [[x[0] + dx, x[1] + dy] for (x, dx, dy) in
                         zip(self.sensor_pos_from_center, dx_from_sensor, dy_from_sensor)]
        return np.array(obstacles_pos)

    def rotate(self, angle, coords):
        """
        Rotates the coordinates of a matrix by the desired angle
        :param angle: angle in radians by which we want to rotate
        :return: numpy.array() that contains rotated coordinates
        """
        R = np.array(((np.cos(angle), -np.sin(angle)),
                      (np.sin(angle), np.cos(angle))))
        return R.dot(coords.transpose()).transpose()

    def local_occupancy(self, abs_pos, sensor_vals):
        '''
        abs_pos: List of lists that will contain the absolute x,y,theta coordinates of the robot
        sensor_vals:  List of lists that will contain the 7 sensor values
        '''

        abs_pos = np.array(abs_pos)

        local_occupancy_grids = [self.obstacles_pos_from_sensor_vals(x) for x in sensor_vals]

        global_map, overall_thymio_coords = [], []

        for (local_grid, pos) in zip(local_occupancy_grids, abs_pos):
            # Rotate the local occupancy grid
            rotated_grid = self.rotate(pos[2] - math.pi / 2, local_grid)
            rotated_thymio_coords = self.rotate(pos[2] - math.pi / 2, self.thymio_coords)

            # Translate the grid at the position of the Thymio
            obstacles_pos = rotated_grid + np.array([pos[0], pos[1]])
            abs_Thymio_coords = rotated_thymio_coords + np.array([pos[0], pos[1]])

            # Store position of the obstacles and Thymio in the global map
            global_map.append(obstacles_pos)
            overall_thymio_coords.append(abs_Thymio_coords)

        global_map = np.array(np.vstack(global_map))

        plt.figure(figsize=(10, 10))
        plt.plot(abs_pos[:, 0], abs_pos[:, 1])
        plt.scatter(global_map[:, 0], global_map[:, 1], color="r", s=10)

        plt.plot(np.array(abs_pos)[:, 0],
                 np.array(abs_pos)[:, 1], color="r", marker="o")

        for coords in overall_thymio_coords:
            plt.plot(coords[:, 0], coords[:, 1], color="g")

        plt.axis("equal");
        plt.show()

## Import Required Libraries and Functions

To run this code we must first import several libraries. The first three lines of the code cell below are easily recognisable. The Local_Nav and Global_Nav files were written to contain the class objects which control local and global navigation respectively. The sys library is imported as it allows us to change the recursion limit on the operating system which is useful for better image detection and path finding.

In [ ]:
import sys
import cv2 as cv
import numpy as np
from tdmclient import ClientAsync
from Local_Nav import Local_Nav as LN
from Global_Nav import Global_Nav 
sys.setrecursionlimit(5000)

## Set Thymio Parameters and Global Variables

In the code cell below global constants concerning the Thymio are initialised. These constants are used to help calculate Thymio speed, angular velocity of the wheels and set the design or "cruising" speed. Another constant, pixel_to_cm, is also intitialised to convert distances seen on the camera into cm distances. These variables are sent to the Global_Nav class to be used as class attributes, they are intialised here so as to be more easily accesible for modifications to be made.

In [ ]:
r               = 2                                 # radius of the Thymio wheels
L               = 9.5                               # distance between the Thymio wheels
v_max           = 20                                # max linear velocity of the thymio in cm/s
cruising        = 200                               # the design speed of the Thymio in this project in Thymio speed units       
motor_speed_max = 500                               # the max motor speed of the Thymio in Thymio speed units
omega_to_motor  = (motor_speed_max*r)/v_max         # the conversion factor from rad/s to Thymio speed units
thymio_size     = 87                                # Size of the Thymio (pixels)
pixel_to_cm     = 1/7.3

## PID Controller

In order to reach it's goal, the Thymio must convert the difference between it's current position and orientation and that of it's goal into a control action. A PID controller is used here which produces a control action that is proportional to this difference, damped so that the control action does not oscillate due to overshooting and corrected for accumulated errors. This method was chosen over a simpler proportional control as it was found through testing that the damping and error correction provided by the derivative and integral terms are necessary for the Thymio to navigate with higher precision in the limited space available to us. The gains for the controller are initialised in the code cell below along with other variables that are used in the calculation of the control action. The values of **Kp**, **Kd** and **Ki** were found through manual tuning of the controller until a desired output was shown. The **previous_error** is used to calculate the error difference between time steps and hence the damping term. The **error_sum** is used to calculate the accumulated error, hence being the integrative term.

The PIDcontroller function can be found as a class method in the Global_Nav.py file. This function takes as input the angle between a vector that goes from the Thymio to the goal, and a vector aligned with the Thymio's currect orientation. The output of the function is the angular velocity of the left and right wheels of the Thymio which are sent to the Thymio at each timestep. The goal of the function is to minimise the angle, thought of as the error, thus turning the Thymio towards the goal. The error is converted to an angular velocity by applying the gains as described in the equation below. To create the control action, the angular velocity of each wheel is described in terms of a design speed and the angular velocity created by the error. The error term is added or subtracted to the design speed of each wheel so as to have an opposite effect on each wheel thereby causing the Thymio to rotate in order to minimise the error and move the Thymio toward the goal at the design speed. 

<center> $\omega$ = Kp * error + Ki * $\Sigma$ error - Kd * (error - previous_error) </center>


In [1]:
Kp             = 0.7   # Proporcional gain
Ki             = 0.02  # Integral gain
Kd             = 1.131  # Derivative gain
error_sum      = 0     # Integral gain
previous_error = 0     # Derivative error

## Kalman Filter

In complex systems such as mobile robots, the state of the sysem is almost impossible to determine with complete accuracy. Sensor data of the system can be used to provide a better estimation of the system state and typically the data is passed through a filter when the variables of interest cannot be measured directly. In the case of the Thymio, its position and orientation can be reliably found using the camera but, when the camera is obscured or for some reason it does not find the thymio in a given frame, we must rely on the Thymio motor speed sensors to infer the new position and orientation at each timestep. Several types of filter exist but the Kalman filter is widely used in robotics for tracking and prediction applications as it is well suited to systems with uncertainty about an observable dynamic process i.e. the motor speeds. 

The Kalman filter is programmed as a recursive function using the estimated state from the previous time step and the sensor data to estimate the current state. At each time step the state transition matrix **A** is used to predict the state variables at the current time step from their previous values. The state covariance **P**, initialised as **P0**, is also predicted using **A**. The observed sensor data at the current time step is then used to calculate the Kalman gain in conjunction with the error in the measurement **R** and the error in the estimate **Q**. Finally the Kalman gain is used to update the prediction of the state and the covariance towards a belief that lies between the initial prediction and the measurement.

The variables used in the Kalman filter function are initialised in the code cell below and the function itself can be found as a class method in the Global_Nav.py file. The covariance function used to create the covariance matrices can also be found in this file. The state transition matrix **A** is constructed so as to progress the coordinates and orientation of the Thymio by a single timestep given their previous rates of change. The covariance matrices are constructed so that each diagonal element is an error corresponding to the state variable which it relates to. The initial values chosen for Q and P0 were based on from the sample values given in the Week 8 Exercise session on Kalman filter implementation and have proved to be sufficient. The values for R were iteratively tuned.

In [ ]:
ts = 0.1                                                           # time step (s)

A = np.array([[1, 0, 0, 0, 0, 0],     # xdot                       # state transition matrix
              [ts, 1, 0, 0, 0, 0],    # x
              [0, 0, 1, 0, 0, 0],     # ydot
              [0, 0, ts, 1, 0, 0],    # y
              [0, 0, 0, 0, 1, 0],     # phidot
              [0, 0, 0, 0, ts, 1]])   # phi

P0 = Global_Nav.covariance(1000, 1000, 1000, 1000, 1000, 1000)       # state covariance matrix
Q  = Global_Nav.covariance(0.0615, 0.04, 0.0615, 0.04, 0.001, 0.01)  # process noise covariance matrix
R  = Global_Nav.covariance(1, 5, 1, 5, 0.1, 0.1)                     # measurement covariance matrix

#R = Global_Nav.covariance(.0615, 0.025, 0.0615, 0.025, 0.0615, 0.025) # measurement covariance matrix

## Begin Thymio Connection and Initialise the Global Navigation Class Object

Client and node explanation

Global Nav explanation

In [ ]:
client = ClientAsync()
node   = client.aw(client.wait_for_node())
#node   = await client.wait_for_node()

# Define global_nav object with class attributes
GN = Global_Nav(client, node, cruising, ts, Kp, Ki, Kd, error_sum, previous_error, r, L, omega_to_motor, pixel_to_cm, R, Q, A, P0)

## Find Shortest Path

Camera vision is used to find the shortest path from the Thymio to the goal. This is implemented in the get_shortest_path function within the Global_Nav class. This function takes as input an image of the environment and the size of the thymio. First the function locates the thymio within the image using a QR code mounted on its back. The image is then converted into greyscale and the Canny edge detection algorithm is used to convert the image into a virtual environment containing only the contours. The OpenCV function cv.Canny() is used for this purpose as it also reduces noise in the image by applying a 5x5 Gaussian filter before applying the edge detection algorithm itself. The image must be put through the Gaussian filter first as edge detection is highly susceptible to noise. 

Once the image has been converted, the corners of the polygons in the image, which are the obstacles in the environment. It is necessary to wait until after the image is converted before attempting to locate the corners as the sharper edges and clearer colour boundaries allow the function to be more accurate. The Harris corner detection algorithm is used here as it is easily available from the OpenCV library and has been proven to be reasonably reliable.

Having the position of the thymio, the goal and the corners of the obstacles in the image, the shortest path that the thymio should take can be found. Both Dijkstra's and the A* algorithm were considered as they make use of nodes, or in this case corners, to find the shortest path. Ultimately it was decided to use Dijkstra's algorithm as it is conveniently available from the pyvisgraph library. However, since the thymio is considered as a point for the purposes of path finding, the positions of the corners in the image must be augmented to prevent the thymio from colliding with the obstacles. This is simply achieved by scaling the polygons by the thymio size. 

In [ ]:
path, mid_point_back, path_cm = GN.get_shortest_path(thymio_size)

## Inicial States

In [ ]:
thymio_pix = np.array([path[0,0], path[0,1]])
thymio_pose = np.array([path_cm[0,0], path_cm[0,1], 0.0])

X = np.array([      [0.0],     # x_dot0
        [thymio_pose[0]],      # x0
                    [0.0],     # y_dot0
        [thymio_pose[1]],      # y0
                    [0.0],     # phi_dot0
        [thymio_pose[2]]])     # phi0 

## Define the Video Window 

In [ ]:
vid = cv.VideoCapture(1+cv.CAP_DSHOW)
vid.set(cv.CAP_PROP_FRAME_WIDTH, 1920*0.7)
vid.set(cv.CAP_PROP_FRAME_HEIGHT, 1080*0.7)

## Controlling the Thymio and Going to the Goal

Use this cell to describe how the Thymio is controlled in the "main" loop. We might split this into more sections.

In [ ]:
next_point = 1

while(True):
    isTrue, img = vid.read()      
    while (np.abs(X[3]-path_cm[-1,1]) > 3) or (np.abs(X[1]-path_cm[-1,0]) > 3):        
        # define current goal
        goal_cm = np.array([path_cm[next_point,0],path_cm[next_point,1]])
        goal = np.array([path[next_point,0],path[next_point,1]])

        while (np.abs(X[3]-goal_cm[1]) > 3) or (np.abs(X[1]-goal_cm[0]) > 3):
            isTrue, img = vid.read()
            img = img[48:643,154:1150]
            if isTrue:
                node = client.aw(client.wait_for_node()) 
                # node   = await client.wait_for_node()
                client.aw(node.lock_node())
                # await node.lock()

                # Find the error in the orientation--------------------------------------------------------------------------------------------------------------
                # vector from the mid back point of the thymio to the current goal
                thymio2goal = goal - mid_point_back 

                # vector from the mid back point of the thymio to the centre of the thymio
                orientation_vec = thymio_pix - mid_point_back
                
                # Find angle through dot product
                norm_1 = np.linalg.norm(orientation_vec)
                norm_2 = np.linalg.norm(thymio2goal)
                dot_product = np.dot(orientation_vec , thymio2goal)
                error = np.arccos(dot_product/(norm_1*norm_2))

                # Find whether the angle is positive or negative
                d = (thymio_pix[0]-mid_point_back[0])*(goal[1]-mid_point_back[1])-(thymio_pix[1]-mid_point_back[1])*(goal[0]-mid_point_back[0])
                if d > 0 :
                    error = -error
                
                # Control action to make the Thymio follow the reference--------------------------------------------------------------------------------------------
                vr, vl = GN.PIDcontroller(error)
                
                motor_left = vl*omega_to_motor
                motor_right = vr*omega_to_motor  
               
                #Activate Obstacle avoidance if necessary
                local_nav = LN(client, node, cruising, cruising)
                flag_obs = local_nav.analyse_data()
                if flag_obs == 1:
                    task = local_nav.obstacle_avoidance() 
                else:
                    adjust_speed = GN.motors(int(motor_left), int(motor_right))
                    node.send_set_variables(adjust_speed)
                node.flush()    
                
                # Update states---------------------------------------------------------------------------------------------------------------------------------------
                motor_sensor = GN.get_motor_speed()
                speed_sensor = motor_sensor/GN.omega_to_motor      # cm/s
                thymio_pix_new,thymio_pose,_,flag,mid_point_back_new = GN.find_thymio(img)
                if flag==1: # if the camera can find the thymio
                    X[1] = thymio_pose[0]                                          # x
                    X[3] = thymio_pose[1]                                          # y
                    X[5] = error                                                   # phi
                    X[4] = (r/L)*(speed_sensor[1]-speed_sensor[0])                 # phi_dot
                    X[0] = (r/2)*(speed_sensor[0]+speed_sensor[1])*np.cos(X[5])    # x_dot
                    X[2] = (r/2)*(speed_sensor[0]+speed_sensor[1])*np.sin(X[5])    # y_dot
                    thymio_pix = thymio_pix_new
                    mid_point_back = mid_point_back_new
                else:
                    X[4] = (r/L)*(speed_sensor[1]-speed_sensor[0])               # phi_dot
                    X[5] = error                                                 # phi
                    X[0] = (r/2)*(speed_sensor[0]+speed_sensor[1])*np.cos(X[5])  # x_dot
                    X[2] = (r/2)*(speed_sensor[0]+speed_sensor[1])*np.sin(X[5])  # y_dot
                    X[1] += X[0]*ts                                              # x
                    X[3] += X[2]*ts                                              # y
                    X = GN.kalman_filter(X)
                    thymio_pix[0] = X[1]/pixel_to_cm
                    thymio_pix[1] = X[3]/pixel_to_cm
                    mid_point_back[0] += X[0]*ts 
                    mid_point_back[1] += X[1]*ts

                # Show thymio and goal position -------------------------------------------------------------------------------------------------------------------------
                img = cv.circle(img, (int(thymio_pix[0]), int(thymio_pix[1])), radius=10, color=(0, 0, 255), thickness=-1)
                img = cv.circle(img, (int(goal[0]), int(goal[1])), radius=10, color=(255, 0, 0), thickness=-1)
                
                cv.imshow("Video", img)
                # the 'q' button is set as the quitting button you may use any desired button of your choice
                if cv.waitKey(1) & 0xFF == ord('q'):
                    break
        
        # Go to next point in path-----------------------------------------------------------------------------------------------------------------------------------------
        next_point += 1
        cv.imshow("Video", img)
        # the 'q' button is set as the quitting button you may use any desired button of your choice
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Stop the thymio when it reaches the end-------------------------------------------------------------------------------------------------------------------------------
    motor_left = 0
    motor_right = 0
    adjust_speed = GN.motors(motor_left, motor_right)
    node.send_set_variables(adjust_speed)
    node.flush()
    img = img[48:643,154:1150]
    cv.imshow("Video", img)
    # the 'q' button is set as the quitting button you may use any desired button of your choice
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release the cap object
vid.release()
# Destroy all the windows
cv.destroyAllWindows()